In [1]:
#thanks this is ripped from https://github.com/thu-vu92/local-llms-analyse-finance/blob/main/categorize_expenses_with_validation.ipynb
from langchain_community.llms import Ollama
from langchain.chains.conversation.memory import ConversationBufferMemory
import pandas as pd
import numpy as np
import os
debugging=1
if debugging:
    file_path = 'd_out.txt'
    f = open(file_path,'w')
    f.write("dubug log")

In [2]:
from os import listdir
from os.path import isfile, join
directory = './transactions/'
transfiles = [directory+f for f in listdir(directory) if isfile(join(directory, f))]
skip_list=['checking','savings']

cc_files=[f for f in transfiles if all(skip not in f for skip in skip_list) and 'lock' not in f]
if debugging: f.write(','.join(cc_files))
    
acc_files = [join(directory, f) for f in listdir(directory) 
         if os.path.isfile(os.path.join(directory, f)) and (f.startswith('checking') or f.startswith('savings'))]
if debugging: f.write(','.join(acc_files))

In [3]:
def read_cc_transactions(transfile):
    df = pd.read_csv(transfile)
    df['account']=transfile.split('_')[1].split('.')[0]
    df['fname']=transfile.split('/')[2].split('.')[0]
    df = df.rename(columns={'Payee': 'Description', 'Posted Date':'Date'})
    
    #https://www.statology.org/pandas-remove-special-characters/
    pattern = r'[^\w\s]'
    df['Description'] = df['Description'].str.replace(pattern, '', regex=True)
    df['Description'] = df['Description'].str.replace(r'\s+', ' ', regex=True)

    return df

In [4]:
def read_acc_transactions(transfile):
    df = pd.read_csv(transfile, delimiter=',')
    df['account']=transfile.split('_')[1].split('.')[0]
    df['fname']=transfile.split('/')[2].split('.')[0]
    #
    if (np.isin([True], df.columns.str.contains('^Unnamed'))):
        df.rename( columns={'Unnamed: 4':'bastard'}, inplace=True )
        mask = df["bastard"].isna() 
        df.loc[~mask, "Amount"] = df.loc[~mask, "Running Bal."]
        df.drop(columns=['bastard'], inplace=True)
        df.dropna(inplace=True)

    df
    
    
    df.drop(columns=['Running Bal.'], inplace=True)
    df['Amount'] = df['Amount'].astype(float)
    # create a new column 'hashed' as a hash of columns 'A' and 'B'
    df['Reference Number'] = df.apply((lambda x: str(hash(str(df['Date'])+str(df['Description'])+str(df['Amount'])+str(df['account'])))), axis=1)

    #https://www.statology.org/pandas-remove-special-characters/
    pattern = r'[^\w\s]'
    df['Description'] = df['Description'].str.replace(pattern, '', regex=True)
    df['Description'] = df['Description'].str.replace(r'\s+', ' ', regex=True)

    return df

In [5]:
cc_transframes=[]
if cc_files:
    for cc_file in cc_files[1:]:
        cc_transframes.append(read_cc_transactions(cc_file))
    cc_trans=pd.concat(cc_transframes, ignore_index=True)

In [6]:
acc_transframes=[]
if acc_files:
    for acc_file in acc_files[1:]:
        acc_transframes.append(read_acc_transactions(acc_file))
    acc_trans=pd.concat(acc_transframes, ignore_index=True)



In [7]:
if 'cc_trans' in locals() and 'acc_trans' in locals():
    if debugging: f.write("merge cc and acc")
    all_trans = cc_trans.merge(acc_trans, on=['Date','Reference Number','Description','Amount','account','fname'],how='outer')
elif 'cc_trans' in locals():
    if debugging: f.write("cc only")
    all_trans = cc_trans
elif 'acc_trans' in locals():
    if debugging: f.write("acc only")
    all_trans = acc_trans
else:
    if debugging: f.write("no files found")
    raise Exception("no files")

if debugging: 
    cc_trans.to_csv('cc_trans.csv', index=False)
    acc_trans.to_csv('acc_trans.csv', index=False)
    all_trans.to_csv('all_trans_merged.csv', index=False)

In [8]:
# Get unique transactions in the Name / Description column
unique_transactions = all_trans["Description"].unique()
unique_transactions_df = pd.DataFrame(unique_transactions,columns=['col1'])


In [9]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

#index_list = list(hop(0, len(unique_transactions), 10))
index_list = list(hop(0, len(all_trans), 10))

In [10]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."


In [11]:
def check_trans(categories_df,all_trans,cnt_trans):
    #print('check_trans')
    
    common_trans= pd.DataFrame()
    common_trans = categories_df['Transaction'].str.lower().isin(all_trans['Description'].str.lower())

    if np.any(~common_trans):
        if debugging: f.write("fail on cat vs all trans")
        return False

    
    if cnt_trans != len(categories_df.index):
        if debugging: f.write("fail on len")
        return False

    if debugging: f.write("passing true")
    return True
    


In [12]:
def categorize_transactions(transaction_names):
    #print("categorzing...")
    llm = Ollama(model="llama3test:8b")
    
    llm.__init__
    response = llm.invoke("""
    Provide a category for a list of transactions. 
    some expenses include just the business name, some include the name and part or all of their address, and some include a confirmation number.
    For example I will privde :
    CVSPHARMACY 123 Somehwer NJ, Spotify AB by Adyen, Beta Boulders Ams Amsterdam Nl,Zelle payment to Bob Manperson Conf o74gqbnm01, CITY OF GOTHAM NJ BILL PAYMENT
    Your response should follow these rules:
    No further explanation or any other text outside of the transactions and their corresponding categories. 
    Do not prepend the answer.
    Do not edit the transaction text in any way. 
    Do not add spacing if there are words together. 
    Do not add or remove any letters. 
    Do not spell check or alter the text in anyway. 
    Do not remove the confirmation id from the string. 
    transactions are separated by ,
    The response should be formatted as follows, with one entry and category per line:
    CVSPHARMACY 123 Somehwer NJ - Pharmacy
    Spotify AB by Adyen - Subscription/Music/Entertainment
    Beta Boulders Ams Amsterdam Nld - Gym/Sports
    Zelle payment to Bob Manperson Conf o74gqbnm01 - Zelle Payment/Misc
    CITY OF GOTHAM NJ BILL PAYMENT - Utilities
    Here is my list of transactions:""" + transaction_names)
    response = response.split('\n')
    if debugging: 
        f.write('transaction_names\n')
        f.write(transaction_names)
        f.write('\n')
        f.write('response\n')
        for resp in response:
            f.write(resp)
            f.write('\n')
    
    ResponseChecks(data = response)

    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.strip().str.split(' - ', expand=True)
    #print(categories_df)    
    return categories_df, response

In [13]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
   
# Loop through the index_list
for i in range(0, len(index_list)-1):
    fcnt=0
    passed=0
    cnt=0
    while(not passed and fcnt <3 and cnt < 4):
        categories_df= pd.DataFrame()
        transaction_names_lst = unique_transactions[index_list[i]:index_list[i+1]]
        transaction_names = ','.join(transaction_names_lst)
        cnt = cnt +1
        
        if len(transaction_names_lst):
            categories_df,response = categorize_transactions(transaction_names)
            if check_trans(categories_df,all_trans,len(transaction_names_lst)):
               passed = 1
            else:
                fcnt= fcnt + 1
                
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [14]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()

In [15]:
# Drop NA values
categories_df_all_no_na = categories_df_all.dropna()

In [16]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
all_trans = pd.merge(all_trans, categories_df_all_no_na, left_on='Description', right_on='Transaction', how='left')


In [17]:
all_trans.to_csv('all_trans_end.csv', index=False)